# Ngram

In [1]:
import torch

In [2]:
print(torch.__version__)

0.3.1.post2


In [3]:
torch.manual_seed(1)

In [4]:
word_to_ix = {'hello':0, 'world':1}
embeds = torch.nn.Embedding(2, 5)
lookup_tensor = torch.LongTensor([word_to_ix['hello']])
hello_embed = embeds(torch.autograd.Variable(lookup_tensor))
print(hello_embed)

Variable containing:
 0.6614  0.2669  0.0617  0.6213 -0.4519
[torch.FloatTensor of size 1x5]



In [5]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10

test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

In [6]:
trigrams = [([test_sentence[i], test_sentence[i+1]], test_sentence[i+2])
           for i in range(len(test_sentence) - 2)]

In [7]:
vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}

In [8]:
class NgramLanguageModeler(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NgramLanguageModeler, self).__init__()
        self.embeddings = torch.nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = torch.nn.Linear(context_size*embedding_dim, 128)
        self.linear2 = torch.nn.Linear(128, vocab_size)
        
    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = torch.nn.functional.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = torch.nn.functional.log_softmax(out, dim=1)
        return log_probs

In [9]:
losses = []
loss_function = torch.nn.NLLLoss()
model = NgramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [10]:
trigrams[:3]

[(['When', 'forty'], 'winters'),
 (['forty', 'winters'], 'shall'),
 (['winters', 'shall'], 'besiege')]

In [11]:
for epoch in range(10):
    total_loss = torch.Tensor([0])
    for context, target in trigrams:
        context_idxs = [word_to_ix[w] for w in context]
        context_var = torch.autograd.Variable(torch.LongTensor(context_idxs))
        
        model.zero_grad()
        
        log_probs = model(context_var)
        
        loss = loss_function(
                log_probs,
                torch.autograd.Variable(torch.LongTensor([word_to_ix[target]]))
        )
        
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.data
    losses.append(total_loss)
print(losses)

[
 522.8848
[torch.FloatTensor of size 1]
, 
 520.2854
[torch.FloatTensor of size 1]
, 
 517.7062
[torch.FloatTensor of size 1]
, 
 515.1464
[torch.FloatTensor of size 1]
, 
 512.6052
[torch.FloatTensor of size 1]
, 
 510.0827
[torch.FloatTensor of size 1]
, 
 507.5763
[torch.FloatTensor of size 1]
, 
 505.0848
[torch.FloatTensor of size 1]
, 
 502.6088
[torch.FloatTensor of size 1]
, 
 500.1473
[torch.FloatTensor of size 1]
]


# CBOW

In [12]:
CONTEXT_SIZE = 2
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules called a
program. People create programs to direct processes. In effect, we
conjure the spirits of the computer with our spells.""".split()

vocab = set(raw_text)
vocab_size = len(vocab)

word_to_ix = {word: i for i, word in enumerate(vocab)}
data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i-2], raw_text[i-1],
              raw_text[i+1], raw_text[i+2]]
    target = raw_text[i]
    data.append((context, target))
print(data[:5])

[(['We', 'are', 'to', 'study'], 'about'), (['are', 'about', 'study', 'the'], 'to'), (['about', 'to', 'the', 'idea'], 'study'), (['to', 'study', 'idea', 'of'], 'the'), (['study', 'the', 'of', 'a'], 'idea')]


In [13]:
class NgramLanguageModeler(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NgramLanguageModeler, self).__init__()
        self.embeddings = torch.nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = torch.nn.Linear(context_size*embedding_dim, 128)
        self.linear2 = torch.nn.Linear(128, vocab_size)
        
    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = torch.nn.functional.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = torch.nn.functional.log_softmax(out, dim=1)
        return log_probs

In [14]:
class CBOW(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(CBOW, self).__init__()
        self.embeddings = torch.nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = torch.nn.Linear(2*context_size*embedding_dim, 128)
        self.linear2 = torch.nn.Linear(128, vocab_size)
        
    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = torch.nn.functional.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = torch.nn.functional.log_softmax(out, dim=1)
        return log_probs
    
def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    tensor = torch.LongTensor(idxs)
    return torch.autograd.Variable(tensor)

In [15]:
make_context_vector(data[0][0], word_to_ix)

Variable containing:
 14
 18
 38
 25
[torch.LongTensor of size 4]

In [17]:
losses = []
loss_function = torch.nn.NLLLoss()
model = CBOW(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

for epoch in range(10):
    total_loss = torch.Tensor([0])
    for context, target in data:
        context_var = make_context_vector(context, word_to_ix)
        
        model.zero_grad()
        
        log_probs = model(context_var)
        
        loss = loss_function(
            log_probs,
            torch.autograd.Variable(torch.LongTensor([word_to_ix[target]]))
        )
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.data
    losses.append(total_loss)
print(losses)

[
 227.4489
[torch.FloatTensor of size 1]
, 
 226.1456
[torch.FloatTensor of size 1]
, 
 224.8502
[torch.FloatTensor of size 1]
, 
 223.5607
[torch.FloatTensor of size 1]
, 
 222.2784
[torch.FloatTensor of size 1]
, 
 221.0024
[torch.FloatTensor of size 1]
, 
 219.7319
[torch.FloatTensor of size 1]
, 
 218.4669
[torch.FloatTensor of size 1]
, 
 217.2054
[torch.FloatTensor of size 1]
, 
 215.9500
[torch.FloatTensor of size 1]
]
